In [1]:
!git clone https://github.com/Allliance/semantic-search-engine
%cd semantic-search-engine/data

Cloning into 'semantic-search-engine'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 1), reused 12 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 2.00 MiB | 12.38 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [7]:
!pip install -r requirements.txt

INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install -r requirements.txt (line 3) and torch because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch
    clip-by-openai 1.1 depends on torch<1.7.2 and >=1.7.1
    The user requested torch
    clip-by-openai 1.0.1 depends on torch<1.7.2 and >=1.7.1
    The user requested torch
    clip-by-openai 0.1.1.5 depends on torch==1.7.1
    The user requested torch
    clip-by-openai 0.1.1.4 depends on torch==1.7.1
    The user requested torch
    clip-by-openai 0.1.1.3 depends on torch==1.7.1
    The user requested torch
    clip-by-openai 0.1.1.2 depends on torch==1.7.1
    The user requested torch
    clip-by-open

In [11]:
!pip install pinecone-client

  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.7 MB/s eta 0:00:00


In [18]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
# from dotenv import load_dotenv
import os

def create_index(index_name="products-index",
                 dimension=512):


    pc = Pinecone(
            api_key="pcsk_5q2j4m_CKKRfFmwKj24tNiiQ6d3uq2bWxqVSYHHWaCRbnSi2VUCc3WDJZQgRHN5Jk2kfSg"
        )

    if index_name not in pc.list_indexes().names():
        pc.create_index(index_name,
                        dimension=dimension,
                        metric="cosine",
                        spec=ServerlessSpec(
                        cloud='aws',
                        region='us-east-1',
                        ))

    return pc

In [47]:

def encode_text_with_processor(text):
    # Preprocess the text using CLIPProcessor
    inputs = preprocess(text=text, return_tensors="pt", padding=True)

    # Send inputs to device (CUDA or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get the text embedding from the CLIP model
    with torch.no_grad():
        text_features = model.get_text_features(**inputs)

    # Normalize the text features
    text_features /= text_features.norm(p=2, dim=-1, keepdim=True)

    return text_features

In [46]:
import pinecone
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
from torchvision import transforms
import torch
import os
import numpy as np
import matplotlib.pyplot as plt # type: ignore

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

index_name = "test-index"
host = "https://test-index-cw4v01q.svc.aped-4627-b74a.pinecone.io"
pc = create_index(index_name)
# Load test images and their embeddings


def load_image_and_embedding(image_paths):
    images = [Image.open(image_path) for image_path in image_paths]
    # image = Image.open(image_path)
    inputs = preprocess(images=images, return_tensors="pt", padding=True).to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)

    # Normalize the image features
    image_features /= image_features.norm(p=2, dim=-1, keepdim=True)

    return image_features, inputs['pixel_values']

def encode_text_with_processor(text):
    # Preprocess the text using CLIPProcessor
    inputs = preprocess(text=text, return_tensors="pt", padding=True)

    # Send inputs to device (CUDA or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get the text embedding from the CLIP model
    with torch.no_grad():
        text_features = model.get_text_features(**inputs)

    # Normalize the text features
    text_features /= text_features.norm(p=2, dim=-1, keepdim=True)

    return text_features

# show the image using plt
def show_image(image):
    image = image.squeeze(0)
    image = image.permute(1, 2, 0)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def upsert_embeddings(product_ids, embeddings):
    # Convert embeddings to a list of tuples (id, vector)
    vectors = [{
        "id": str(product_id),
        "values": embedding.tolist(),
        "metadata": {},
        } for product_id, embedding in zip(product_ids, embeddings)]

    # Connect to the index
    index = pc.Index(index_name)#, host=host)

    # Upsert the embeddings
    index.upsert(vectors)

image_files = {
    'cat': 'test_images/cat.png',
    'dog': 'test_images/dog.png',
    'elephant': 'test_images/elephant.jpg',
}

names = image_files.keys()
image_paths = [image_files[name] for name in names]

embeddings, images = load_image_and_embedding(image_paths)

print(len(embeddings.tolist()))
# print(image.size())
# show_image(image)

# Upsert the image embedding
upsert_embeddings(names, embeddings)

3


In [60]:
query = "persian"
query_embedding = encode_text_with_processor(query).squeeze(0)

index = pc.Index(index_name)

response = index.query(
    vector=query_embedding.tolist(),
    top_k=1,
    include_values=True,
    # include_metadata=True,
    # filter={"genre": {"$eq": "action"}}
)

print(response['matches'][0]['id'], response['matches'][0]['score'])

cat 0.235475332


In [49]:
query_embedding.size()

torch.Size([1, 512])